# 研究データの容量をモニタリングする

研究データ容量について、DMPに記載していただいた上限を超過していないかモニタリングします。  
以下のセルを上から実行してください。

## 1. 基準値となるデータ容量上限値を取得する

In [ ]:
import os

os.chdir(os.environ['HOME'])
# リポジトリと同期して内容を最新化する
!datalad update -s gin --merge

In [ ]:
import os
import json

path_flow_root = '/home/jovyan/WORKFLOWS/FLOW/'
os.chdir(path_flow_root)
from util.scripts import utils 

path_params = utils.fetch_monitoring_param_file_path()
limit = ""
with open(path_params, "r") as f:
    dmp = json.load(f)
    limit = dmp["dataSize"]

In [ ]:
print("input : " + str(limit))

if isinstance(limit, str):  # "100", "100B", "100b", "100KB", "100kb"
    limit = limit.lower()

    if limit[-1] == "b":
        limit = limit[:-1]

    units = ["k", "m", "g", "t", "p", "e", "z"]
    if limit[-1] in units:
        limit = 1024 ** (units.index(limit[-1]) + 1) * int(limit[:-1])

print("output: " + str(limit))


## 2. 実験パッケージ情報を取得する

In [ ]:
import os
import json
import glob

# 作業ディレクトリに移動
os.chdir(os.environ['HOME'] + '/experiments/')

# 実験パッケージ群の読み込み
dirs = glob.glob("./*/")

experiments = []
for dir_name in dirs:
    experiments += [os.path.basename(dir_name[:-1])]
print("実験パッケージ　：" + str(experiments))

## 3. 各実験パッケージについてデータの実体を取得する

In [ ]:
import os
for experiment in experiments:
    os.chdir(os.environ['HOME'] + '/experiments/' + experiment)
    !git pull
    !datalad get -r .

## 4. データ容量をチェックする

In [ ]:
isSucceeded = True

result = !du -s ~/experiments/
result = result[0].split("\t")[0]
result = int(result)
print("datasize: " + str(result))

if result > limit:
    isSucceeded = False
    raise Exception("データ容量を超過しています")

## 5. モニタリング結果をREADMEに反映する

In [ ]:
os.chdir(path_flow_root)
from util.scripts import utils

utils.reflect_monitoring_results('dataAmount', isSucceeded, '/home/jovyan')

## 6. 本タスクの実行結果をGIN-forkに同期する

ここまでの内容を保存し、GIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](http://dg02.dg.rcos.nii.ac.jp/G-Node/Info/wiki/%E3%83%AF%E3%83%BC%E3%82%AF%E3%83%95%E3%83%AD%E3%83%BC#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
import papermill as pm
from colorama import Fore
from IPython.display import clear_output

os.chdir(os.environ['HOME'])
save_path = ['/home/jovyan/README.md', '/home/jovyan/WORKFLOWS/FLOW/02_experimental_phase/base_monitor_data_size.ipynb']
try:
    pm.execute_notebook(
        'WORKFLOWS/FLOW/util/base_datalad_save_push.ipynb',
        '/home/jovyan/.local/push_log.ipynb',
        parameters = dict(SAVE_MESSAGE = 'モニタリング（研究データ容量）', IS_RECURSIVE = False, TO_GIT = True, PATH = save_path)
    )
finally:
    clear_output()
    %store -r DATALAD_MESSAGE
    %store -r DATALAD_ERROR
    print('\n' + DATALAD_MESSAGE + '\n')
    print(Fore.RED + DATALAD_ERROR)

## 7. 研究ワークフロー機能トップページに遷移する

続けてワークフロー機能を実行する場合は、[こちら](../../base_FLOW.ipynb)から遷移できます。